In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

path = "../input/tomato/New Plant Diseases Dataset(Augmented)"
os.listdir(path)

In [ ]:
train_path = os.path.join(path, "train")
print(os.listdir(train_path))
print("*"*100)
test_path = os.path.join(path, "valid")
print(os.listdir(test_path))

In [ ]:
from glob import glob
folders = glob("kaggle/input/tomato/New Plant Diseases Dataset(Augmented)/train/*")
folders

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(plt.imread("../input/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Bacterial_spot/00416648-be6e-4bd4-bc8d-82f43f8a7240___GCREC_Bact.Sp 3110.JPG"))
plt.title("Bacterial Spot")

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import *
from tensorflow.keras.initializers import *
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
SIZE = [124, 124]
BATCH_SIZE = 32
num_classes = 2
learning_rate = 0.0001

In [ ]:

train_datagen_mnetv2 = ImageDataGenerator(rescale=1./255)
test_datagen_mnetv2 = ImageDataGenerator(rescale=1./255)

In [ ]:
seed =random.randint(1, 1000)

In [ ]:
training_set_vg19 = train_datagen_vg19.flow_from_directory(train_path,
                                                 target_size=(124, 124),
                                                 batch_size=16,
                                                 class_mode="categorical", shuffle=True)


In [ ]:

testing_set_vg19 = test_datagen_vg19.flow_from_directory(test_path,
                                                 target_size=(124, 124),
                                                 batch_size=16,
                                                 class_mode="categorical", shuffle=False)

In [ ]:

from tensorflow import keras
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=SIZE + [3])

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(rate = .4)(x)
x = BatchNormalization()(x)
x = Dense(1280, activation='relu',  kernel_initializer=glorot_uniform(seed))(x)
x = Dropout(rate = .4)(x)
x = BatchNormalization()(x)



# Freezing MobileNet layers
for layer in base_model.layers:
    layer.trainable=False
x = Flatten()(base_model.output)   
predictions = Dense(10, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)
optimizer = Adam(lr=learning_rate)

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(training_set_vg19.classes), 
            testing_set_vg19.classes)
class_weights

In [ ]:
nb_train_samples = training_set_vg19.samples
nb_validation_samples = testing_set_vg19.samples

In [ ]:
trainning_set_vg19 = train_datagen_vg19.flow_from_directory(train_path,
                                                 target_size=(124, 124),
                                                   batch_size=16,
                                               class_mode="categorical", shuffle=True)

In [ ]:
epochs=20
import tensorflow as tf
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor='val_loss', save_best_only=True, verbose=1),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=50,verbose=1)
]

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history=model.fit_generator(training_set_vg19,
                       validation_data=testing_set_vg19,
                       epochs=20,
                       callbacks=[callback],
                     )
  #  callbacks=[callback]   ,    ,steps_per_epoch=len(trainning_set_vg19),validation_steps=nb_validation_samples,steps_per_epoch=nb_train_samples, class_weight = class_weights
                     #  validation_steps=len(testing_set_vg19),

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history['val_accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))
plt.title("MobileNetv2")
plt.plot(epochs, accuracy, "r", label="trainning accuracy")
plt.plot(epochs, val_accuracy, "r", label="validation accuracy")
plt.legend()
plt.show()

plt.plot(epochs, loss, "b", label="trainning loss")
plt.plot(epochs, val_loss, "r", label="validation loss")
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
# Load the best saved model

Y_pred = model.predict_generator(testing_set_vg19, nb_test_samples // BATCH_SIZE+1)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
y_pred = np.argmax(y_pred, axis=1)
y_pred


In [ ]:
model.save("vgg_19tl.model")

In [ ]:
from tensorflow import lite

In [ ]:
converter = lite.TFLiteConverter.from_keras_model(modelvg)
tfmodel = converter.convert()

tfmodel_size = len(tfmodel) / 1024
print('tfmodel size = %dKBs.' % tfmodel_size)

#open('tomato.tflite', 'wb').write(tfmodel)
#modelvg.save("tomato.tflite")

In [ ]:
open('tomato.tflite', 'wb').write(tfmodel)

f.write(tfmodel)
f.close()